# Work with json and path

## Part 2

In [1]:
import os
import shutil
import glob
import fnmatch
import json
from nltk import FreqDist
import csv

#### move all restaurants json files to a new folder called Restaurants

In [2]:
def create_folder(fdr_path, fdr_name):
    os.mkdir(os.path.join(fdr_path, fdr_name))

In [3]:
qpath = '/Users/Rui/Documents/INFO 7374/Assignment2/'

In [4]:
create_folder(qpath, 'Restaurants')

In [5]:
create_folder(qpath+'Restaurants','json files')

In [6]:
workpath = '/Users/Rui/Documents/INFO 7374/Assignment2/Restaurants/json files'
rpath = '/Users/Rui/Documents/INFO 7374/Assignment 2 - Requirements/Data/'

In [7]:
allfiles = glob.glob(rpath+'*')

for file in allfiles:
    ft = json.loads(open(file).read())['term']
    if ft=='restaurants':
        shutil.copy(file, workpath)

In [8]:
allfiles = glob.glob(workpath+'/*')
print(len(allfiles))

6372


#### create a csv file under Restaurants folder to for output and a function for writing to file

In [9]:
with open((qpath+'Restaurants/'+'RestaurantsOpenHours.csv'),'w+',newline='') as fp:
    fieldnames = ['Name of restaurant','City','Country code','Day of week','Start time hour','Start time minute','End time hour','End time minute']
    writer = csv.DictWriter(fp, fieldnames=fieldnames)
    writer.writeheader() 

In [10]:
def write_to_file(res_list):  #res_list is a list of dictionaries
    with open((qpath+'Restaurants/'+'RestaurantsOpenHours.csv'),'a',newline='') as fp:   
        fieldnames = ['Name of restaurant','City','Country code','Day of week','Start time hour','Start time minute','End time hour','End time minute']
        writer = csv.DictWriter(fp, fieldnames=fieldnames)        
        a = 0
        while a < len(res_list):
            writer.writerow(res_list[a])
            a = a + 1

#### try some examples...

In [11]:
print(json.loads(open(allfiles[0]).read())['name'])
print(json.loads(open(allfiles[0]).read())['location']['city'])
print(json.loads(open(allfiles[0]).read())['location']['country'])
print(json.loads(open(allfiles[0]).read())['hours'][0]['open'])
print(json.loads(open(allfiles[0]).read())['hours'][0]['open'][0])
print(json.loads(open(allfiles[0]).read())['hours'][0]['open'][0]['day'])
print(json.loads(open(allfiles[0]).read())['hours'][0]['open'][0]['end'][:2])
print(json.loads(open(allfiles[0]).read())['hours'][0]['open'][0]['end'][2:])
print(json.loads(open(allfiles[0]).read())['hours'][0]['open'][0]['start'][:2])
print(json.loads(open(allfiles[0]).read())['hours'][0]['open'][0]['start'][2:])

1 Abercrombie Lane
Sydney
AU
[{'day': 0, 'end': '1530', 'is_overnight': False, 'start': '0730'}, {'day': 1, 'end': '1530', 'is_overnight': False, 'start': '0730'}, {'day': 2, 'end': '1530', 'is_overnight': False, 'start': '0730'}, {'day': 3, 'end': '1530', 'is_overnight': False, 'start': '0730'}, {'day': 4, 'end': '1530', 'is_overnight': False, 'start': '0730'}]
{'day': 0, 'end': '1530', 'is_overnight': False, 'start': '0730'}
0
15
30
07
30


#### country and city are two common attributes shared by many files, thus I categorized allfiles into subfolders accordingly

In [12]:
wp = workpath +'/'
country_list = []
for file in glob.glob(wp+'*'):
    country_list.append(json.loads(open(file).read())['location']['country'])
country_list = FreqDist(country_list).most_common(100)
for country in country_list:
    create_folder(wp, country[0])

In [13]:
for file in glob.glob(wp+'*'):
    if (os.path.isfile(file) == True):
        fc = json.loads(open(file).read())['location']['country']
        shutil.move(file, wp + fc + '/' + os.path.basename(file))

In [14]:
for folder in glob.glob(wp+'*'):
    city_list =[]
    for file in glob.glob(folder + '/*'):
        if (os.path.isfile(file) == True):
            city_list.append(json.loads(open(file).read())['location']['city'].title())
    city_list = FreqDist(city_list).most_common(100)
    for city in city_list:
        create_folder(folder+'/', city[0])

In [15]:
for folder in glob.glob(wp+'*'):
    for file in glob.glob(folder + '/*'): 
        if (os.path.isfile(file) == True):
            fc = json.loads(open(file).read())['location']['city']
            shutil.move(file, folder+'/'+fc+'/'+os.path.basename(file))     

Dictionary must looks like this...<br>
{'Name of restaurant':name,'City':city,'Country code':country,'Day of week':day,'Start time hour':s_hour,'Start time minute':s_min,'End time hour':e_hour,'End time minute':e_min}

#### Actual Work
loop through ['open'] dictionary and insert into my csv file

In [16]:
for country_folder in glob.glob(workpath+'/*'):
    country = os.path.basename(country_folder)
    for city_folder in glob.glob(country_folder+'/*'):
        city = os.path.basename(city_folder)
        for file in glob.glob(city_folder+'/*'):
            file_list = []
            name = json.loads(open(file).read())['name']
            if 'hours' in json.loads(open(file).read()):            
                open_info = json.loads(open(file).read())['hours'][0]['open']
                for ed in open_info:
                    day = ed['day']
                    s_hour = ed['start'][:2]
                    s_min = ed['start'][2:]
                    e_hour = ed['end'][:2]
                    e_min = ed['end'][2:]
                    ed_dict = {'Name of restaurant':name,'City':city,'Country code':country,'Day of week':day,'Start time hour':s_hour,'Start time minute':s_min,'End time hour':e_hour,'End time minute':e_min}
                    file_list.append(ed_dict)
            write_to_file(file_list)

1. the leading 0 for minutes were actually written to csv file but was dropped as soon as the file was opened.<br>
2. one way to solve the problem is to open the file with texteditor, save the file with txt extension.<br>
3. open the txt file again in excel, select first column -> Data-> Text to columns -> Delimited-> Comma-> Text-> Finish<br>
4. then save the file to xlsx...